In [1]:
import os
import numpy as np
import pandas as pd
import altair as alt
import docx

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [3]:
# Used for dated output files
# working_date="260624"
working_date="180424"
# Change to git repo outer directory
os.chdir(os.path.join("..", ".."))

### Aggregate HsMetrics for CHIP-panel blood/saliva pairs

In [4]:
# Parse hsmetrics files for blood/saliva comparison run
runplates = [os.path.join("pipeline_outputs", p) for p in os.listdir("pipeline_outputs") if p.startswith("ARCH_run04")]
take3 = lambda x: '_'.join(x.split('_')[:3])  # Helper function to extract run/plate info from directory name
hsn = "multiqc_picard_HsMetrics.txt"
hsmetrics_report_paths = [(take3(y), os.path.join(x, y, hsn)) for x in runplates for y in os.listdir(x) if y.endswith("hsmetrics_multiqc_report_data")]

In [5]:
# Combine all hsmetrics data into df
hsmdf = pd.concat([pd.read_csv(p, delimiter='\t').assign(runplate=n) for n, p in hsmetrics_report_paths]).reset_index(drop=True)
# Split and expand run/plate data into dedicated columns
hsmdf = hsmdf.join(hsmdf.runplate.str.split('_', expand=True).rename(columns={0: "project", 1: "run", 2: "plate"}))

#### Stratified coverage metrics

In [6]:
# Stratified metrics table
# Extract DNA source information from sample dispatch list
sourcedf = pd.read_excel("CHIP_phase 2_plate_layouts_CHIP samples.xlsx", sheet_name="Dispatchlist")#
sourcemap = sourcedf[sourcedf["Collection Type"].isin({"ABC_SALIVA", "ABC_BLD"})].set_index("Spec ID")["Collection Type"].to_dict()

In [7]:
# Add DNA source information to hsmetrics dataframe
hsmdf = hsmdf.assign(specid=hsmdf.Sample.str.extract(r"(BT\d+)").fillna("CONTROL"))
hsmdf = hsmdf.assign(dna_source=hsmdf.specid.map(sourcemap))

In [8]:
# Adding two strata of pass/fail cutoffs
hsmdf = hsmdf.assign(
    passfail_80_500=hsmdf["PCT_TARGET_BASES_500X"] > 0.8,
    passfail_50_500=hsmdf["PCT_TARGET_BASES_500X"] > 0.5
)

In [9]:
def summarise_subgroups_hsmetrics(hsmdf, group_column):
    bldstats = hsmdf.query("dna_source == 'ABC_BLD'") \
        .groupby(group_column)[["MEAN_TARGET_COVERAGE"]] \
        .agg(["count", "median", "mean", "min", "max"]) \
        .droplevel(0, axis=1).assign(dna_source="BLD")
    salstats = hsmdf.query("dna_source == 'ABC_SALIVA'") \
        .groupby(group_column)[["MEAN_TARGET_COVERAGE"]] \
        .agg(["count", "median", "mean", "min", "max"]) \
        .droplevel(0, axis=1).assign(dna_source="SALIVA")
    horizonstats = hsmdf[hsmdf.Sample.str.startswith("Control_HD829")] \
        .groupby(group_column)[["MEAN_TARGET_COVERAGE"]] \
        .agg(["count", "median", "mean", "min", "max"]) \
        .droplevel(0, axis=1).assign(dna_source="HORIZON")
    hmwstats =  hsmdf[hsmdf.Sample.str.startswith("Control_X4336")] \
        .groupby(group_column)[["MEAN_TARGET_COVERAGE"]] \
        .agg(["count", "median", "mean", "min", "max"]) \
        .droplevel(0, axis=1).assign(dna_source="HMW")
    allstats = pd.concat([bldstats, salstats, horizonstats, hmwstats]) \
        .assign(criteria=f"{group_column.split('_')[1]}%_{group_column.split('_')[2]}X").reset_index() \
        .rename(columns={group_column: "pass_criteria"})
    return allstats
allstats_80_500 = summarise_subgroups_hsmetrics(hsmdf, "passfail_80_500")
allstats_50_500 = summarise_subgroups_hsmetrics(hsmdf, "passfail_50_500")

In [10]:
allstats_allthresholds = pd.concat([allstats_80_500, allstats_50_500]).set_index(["criteria", "dna_source", "pass_criteria"])

In [11]:
format_depth = lambda x: f"{round(x, 2)}X" if isinstance(x, float) else x
allstats_formatted = allstats_allthresholds.map(format_depth)

In [12]:
allstats_formatted

count    median      mean       min  \
criteria dna_source pass_criteria                                        
80%_500X BLD        False              1   242.13X   242.13X   242.13X   
                    True              93  1341.83X  1365.04X   892.58X   
         SALIVA     False             32    741.1X   638.66X     0.48X   
                    True              62  1167.49X  1234.51X   842.97X   
         HORIZON    True               2   919.21X   919.21X   885.43X   
         HMW        True               2   1865.2X   1865.2X  1803.96X   
50%_500X BLD        False              1   242.13X   242.13X   242.13X   
                    True              93  1341.83X  1365.04X   892.58X   
         SALIVA     False              8   176.53X    225.0X     0.48X   
                    True              86  1044.75X   1106.7X   589.74X   
         HORIZON    True               2   919.21X   919.21X   885.43X   
         HMW        True               2   1865.2X   1865.2X  1803.96X   

                                        max  
criteria dna_source pass_criteria            
80%_500X BLD        False           242.13X  
                    True           1909.86X  
         SALIVA     False           974.54X  
                    True           2082.23X  
         HORIZON    True            952.99X  
         HMW        True           1926.44X  
50%_500X BLD        False           242.13X  
                    True           1909.86X  
         SALIVA     False           478.97X  
                    True           2082.23X  
         HORIZON    True            952.99X  
         HMW        True           1926.44X

In [13]:
# Optionally write out hsmetrics stats
# allstats_formatted.to_excel(
#     os.path.join("CHIP-only_blood_saliva_comparisons", f"bldsal_tables_hsmetrics_multistrata_{working_date}.xlsx"),
#     engine="xlsxwriter",
# )

In [14]:
# Getting list of fails for exclusion in comparisons
passfaildf = hsmdf.assign(specid=hsmdf.Sample.str.extract(r"(BT\d+)"))[["specid", "passfail_80_500"]]
fail_ids = set(passfaildf[~passfaildf.passfail_80_500].specid)

### Parse long comparison data, exclude pairs with a fail, exclude artefacts, annotate, summarise in table

In [15]:
chiprange_varstats_path = os.path.join("CHIP-only_blood_saliva_comparisons", "ARCH_bld_sal_chip_only_180424_CHIP-only_flt-chiprange_bed.csv")
chiprange_allvarstats_path = os.path.join("CHIP-only_blood_saliva_comparisons", "ARCH_bld_sal_chip_only_180424_CHIP-only_flt-chiprange_bed_allvarstats.csv")
vsdf = pd.read_csv(chiprange_varstats_path)
avsdf = pd.read_csv(chiprange_allvarstats_path)

In [16]:
# Get independent blood/saliva carrier counts for each unique variant (also calculate percentages, count / 94)
varcounts_bld = avsdf.query("source == 'BLD'")[["varid", "comparison_id"]].groupby("varid").count()
varcounts_bld = varcounts_bld.assign(carrier_pct=varcounts_bld.comparison_id / 94)
varcounts_sal = avsdf.query("source == 'SAL'")[["varid", "comparison_id"]].groupby("varid").count()
varcounts_sal = varcounts_sal.assign(carrier_pct=varcounts_sal.comparison_id / 94)

In [17]:
# Get sets of artefact variant ids based on threshold for both datasets
artefact_threshold = 0.1
artefacts_sal = set(varcounts_sal.query(f"carrier_pct > {artefact_threshold}").index)
artefacts_bld = set(varcounts_bld.query(f"carrier_pct > {artefact_threshold}").index)
# Remove variants failing artefact threshold
avsdf = avsdf[avsdf.varid.isin((artefacts_sal | artefacts_bld))]

### Write out variant data and annotate

In [18]:
# Function to write varid field of idf to minimal vcf format for vep annotation
def write_minimal_vep_vcf(idf, outputname):
    # Split varid into vcf coords
    codf = idf[["varid"]].varid.str.split(':', expand=True).rename(columns={0: "#CHROM", 1: "POS", 2: "REF", 3: "ALT"})
    codf = codf.assign(ID=idf.varid, QUAL='.', FILTER='.', INFO='.', FORMAT='.')
    codf = codf[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"]]
    codf.to_csv(outputname, sep='\t', index=False)
# write_minimal_vep_vcf(avsdf, os.path.join("CHIP-only_blood_saliva_comparisons", f"bld_sal_ten_genes_chiprange_comparison_allcalls_{working_date}.vcf"))

In [19]:
# Run VEP annotation on output VCF here

In [20]:
# Reading annotated output, varid to join on is located in #Uploaded_variation column
avsdf_anno = pd.read_csv(
    os.path.join("CHIP-only_blood_saliva_comparisons", f"bld_sal_ten_genes_chiprange_comparison_allcalls_{working_date}.vep.tsv"),
    skiprows=124, sep='\t'
).rename(columns={"#Uploaded_variation": "varid"}).drop_duplicates()

# Add in vcf/BT names before join
avsdf = avsdf.merge(vsdf[["UPN", "BLD_VCF", "SAL_VCF"]], left_on="comparison_id", right_on="UPN", how="left").drop("UPN", axis=1)
avsdf_anno_joined = avsdf.join(avsdf_anno.set_index("varid"), on="varid", how="left").sort_values("varid")

In [21]:
# Getting list of fail UPNs to filter comparison stats df
crvdf = vsdf.copy(deep=True)
crvdf = crvdf.assign(
    bld_specid=crvdf.BLD_VCF.str.extract(r"(BT\d+)"),
    sal_specid=crvdf.SAL_VCF.str.extract(r"(BT\d+)"),
)
fail_upns = set(crvdf[(crvdf.bld_specid.isin(fail_ids) | crvdf.sal_specid.isin(fail_ids))].UPN)

In [22]:
# Creating comparison stats df from variant info
cstatsdf = avsdf_anno_joined[["comparison_id", "varid", "source"]].assign(value=1) \
    .pivot(index=["comparison_id", "varid"], values="value", columns="source").fillna(0).reset_index()
# Removing UPNs with >= 1 fail in the pair
cstatsdf = cstatsdf[~cstatsdf.comparison_id.isin(fail_upns)]
cstatsdf = cstatsdf.assign(BLD_SAL=cstatsdf.BLD + cstatsdf.SAL)
# Converting concordance to counts by sample pair
cstatsdf = cstatsdf.assign(
    BLD_only=((cstatsdf.BLD_SAL == 1.0) & (cstatsdf.BLD == 1.0)).astype(int),
    SAL_only=((cstatsdf.BLD_SAL == 1.0) & (cstatsdf.SAL == 1.0)).astype(int),
    BLD_SAL=(cstatsdf.BLD_SAL == 2.0).astype(int),
).drop(columns=["BLD", "SAL", "varid"])
cstatsdf = cstatsdf.groupby("comparison_id").sum().reset_index()

In [23]:
# Add percentages
cstatsdf = cstatsdf.assign(
    total_vars=(cstatsdf.BLD_SAL + cstatsdf.BLD_only + cstatsdf.SAL_only),
    pct_BLD_SAL=cstatsdf.BLD_SAL / (cstatsdf.BLD_SAL + cstatsdf.BLD_only + cstatsdf.SAL_only),
    pct_BLD_only=cstatsdf.BLD_only / (cstatsdf.BLD_SAL + cstatsdf.BLD_only + cstatsdf.SAL_only),
    pct_SAL_only=cstatsdf.SAL_only / (cstatsdf.BLD_SAL + cstatsdf.BLD_only + cstatsdf.SAL_only),
)

In [24]:
# Add obfuscated pair ID for tables
cstatsdf = cstatsdf.sort_values("pct_BLD_SAL", ascending=False).assign(pair_id=[f"P{n+1:02d}" for n in range(len(cstatsdf))])
# Order columns for output
conc_tbldf = cstatsdf[["pair_id", "total_vars", "BLD_only", "SAL_only", "BLD_SAL", "pct_BLD_only", "pct_SAL_only", "pct_BLD_SAL"]]

In [25]:
# Write out concordance table
# conc_tbldf.to_csv(os.path.join("CHIP-only_blood_saliva_comparisons", f"bldsal_concordance_table_{working_date}.csv"), index=False)